### Primer Modelo de Detección de Anomalías

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import (
  train_test_split,
  cross_val_score,
  GridSearchCV
)
from sklearn.preprocessing import (
  StandardScaler,
  RobustScaler
)
from sklearn.ensemble import (
  RandomForestClassifier, 
  IsolationForest
)
from sklearn.linear_model import (
  LogisticRegression
)
from xgboost import XGBClassifier
from sklearn.metrics import (
  classification_report, 
  confusion_matrix,
  roc_auc_score,
  roc_curve,
  precision_recall_curve,
  f1_score,
  recall_score, 
  precision_score
)
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Configuración de estilo para las gráficas
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

log_mlab_path = '../data/target/access_log_master_manual_labeling.csv'
df = pd.read_csv(log_mlab_path)

In [ ]:
df['anomaly'].value_counts()

In [ ]:
df_labeled = df[df['anomaly'] != -1]
display(df_labeled['anomaly'].value_counts())
display(df_labeled.shape)

In [ ]:
df_labeled.columns

### Codificar Variables

In [ ]:
from sklearn.preprocessing._label import LabelEncoder

method_series = df_labeled['method'].copy()
method_series = method_series.fillna('UNKNOWN')
method_encoder = LabelEncoder()
df_labeled['method_encoded'] = method_encoder.fit_transform(method_series)

category_mapping = dict(zip(method_encoder.classes_, method_encoder.transform(method_encoder.classes_)))
display(category_mapping)

### Training-Test Model

In [ ]:
features = [
  #'ip_client', 'timestamp', 'status', 
  'size', 
  #'user_agent', 
  #'method',
  'method_encoded',
  #'url', 
  #'protocol', 
  #'anomaly', # target 
  #'status_category', 
  'url__count_sql_words','url__count_xss_words', 'url__count_command_words',
  'url__count_auth_words', 'url__count_error_words',
  'url__count_malware_words', 'url__count_danger_characters',
  'url__count_obfuscation_code_words', 'url__count_dir_words',
  'url__count_dot', 'url__count_http', 'url__count_percentage_symbol',
  'url__count_question_symbol', 'url__count_hyphen', 'url__count_equal',
  'url__url_length', 'url__digit_count', 'url__letter_count',
  'url__count_special_characters', 'url__is_encoded',
  'url__unusual_character_ratio'
]

In [ ]:
X = df_labeled[features]
y = df_labeled['anomaly']

In [ ]:
X.isna().sum()

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(20, 15))
axes = axes.ravel()

for idx, feature in enumerate(features):
  for label in [0, 1]:
    subset = X[y == label][feature]
    if subset.dtype == bool:
      subset = subset.astype(int)
    axes[idx].hist(subset, alpha=0.5, label=f'Class {label}', bins=30)
  axes[idx].set_title(feature)
  axes[idx].legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
correlations = pd.concat([X, y], axis=1).corr()['anomaly'].sort_values(ascending=False)
print(correlations)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
  X, y,
  test_size=0.2,
  random_state=42,
  stratify=y  # Importante para datos desbalanceados
)

In [ ]:
# Usar RobustScaler por si hay outliers
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir de nuevo a DataFrame para mejor visualización
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=features, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=features, index=X_test.index)

### Baseline: Random Forest

In [ ]:
rf_base = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',  # Ajusta pesos automáticamente
    random_state=42,
    n_jobs=-1
)

rf_base.fit(X_train_scaled_df, y_train)

# Predicciones
y_pred_rf = rf_base.predict(X_test_scaled_df)
y_pred_proba_rf = rf_base.predict_proba(X_test_scaled_df)[:, 1]

# Métricas
print("=== Random Forest Base ===")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_rf):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf):.4f}")
print(f"Recall (detección de anomalías): {recall_score(y_test, y_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Normal', 'Anomalía']))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Anomalía'], yticklabels=['Normal', 'Anomalía'])
plt.title('Matriz de Confusión - Random Forest')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.show()


### Modelo con SMOTE (Oversampling)

In [ ]:
smote_pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42, sampling_strategy=0.5)),  # Reducir desbalanceo
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    ))
])

smote_pipeline.fit(X_train_scaled_df, y_train)

# Predicciones
y_pred_smote = smote_pipeline.predict(X_test_scaled_df)
y_pred_proba_smote = smote_pipeline.predict_proba(X_test_scaled_df)[:, 1]

print("=== Random Forest con SMOTE ===")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_smote):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_smote):.4f}")
print(f"Recall (detección de anomalías): {recall_score(y_test, y_pred_smote):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_smote):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_smote, target_names=['Normal', 'Anomalía']))


### XGBoost con Manejo de Desbalanceo

In [ ]:
# Calcular ratio de desbalanceo para XGBoost
scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])  # normal/anomaly

xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,  # Manejo de clases desbalanceadas
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

xgb_model.fit(X_train_scaled_df, y_train)

# Predicciones
y_pred_xgb = xgb_model.predict(X_test_scaled_df)
y_pred_proba_xgb = xgb_model.predict_proba(X_test_scaled_df)[:, 1]

print("=== XGBoost ===")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_xgb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_xgb):.4f}")
print(f"Recall (detección de anomalías): {recall_score(y_test, y_pred_xgb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_xgb):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=['Normal', 'Anomalía']))



### Comparación de Modelos

In [ ]:
# Comparar curvas ROC
plt.figure(figsize=(10, 8))

# Random Forest Base
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.3f})', linewidth=2)

# Random Forest con SMOTE
fpr_smote, tpr_smote, _ = roc_curve(y_test, y_pred_proba_smote)
auc_smote = roc_auc_score(y_test, y_pred_proba_smote)
plt.plot(fpr_smote, tpr_smote, label=f'Random Forest + SMOTE (AUC = {auc_smote:.3f})', linewidth=2)

# XGBoost
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)
auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {auc_xgb:.3f})', linewidth=2)

# Línea de referencia
plt.plot([0, 1], [0, 1], 'k--', label='Clasificador Aleatorio')

plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curvas ROC - Comparación de Modelos')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

### Validación Cruzada Estratificada

In [ ]:
# Agregar importaciones necesarias
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_fscore_support

# Definir las métricas de evaluación
def evaluate_model(model, X, y, cv=5):
    """
    Evalúa un modelo usando validación cruzada estratificada y devuelve métricas.
    
    Args:
        model: Modelo de sklearn con .fit() y .predict_proba()
        X: Características
        y: Etiquetas
        cv: Número de folds (por defecto 5)
    
    Returns:
        Diccionario con métricas promedio
    """
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    
    # Almacenar métricas por fold
    accuracies, aucs, precisions, recalls, f1_scores = [], [], [], [], []
    
    # Predicciones para métricas globales
    y_true_all = []
    y_pred_all = []
    y_pred_proba_all = []
    
    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Aplicar SMOTE solo en el fold de entrenamiento
        smote = SMOTE(random_state=42)
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
        
        # Entrenar modelo
        model.fit(X_train_res, y_train_res)
        
        # Predecir
        y_pred = model.predict(X_val)
        y_pred_proba = model.predict_proba(X_val)[:, 1] if hasattr(model, "predict_proba") else None
        
        # Calcular métricas
        accuracies.append(accuracy_score(y_val, y_pred))
        if y_pred_proba is not None:
            aucs.append(roc_auc_score(y_val, y_pred_proba))
        
        # Métricas por clase (para anomalía=1)
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_val, y_pred, average='binary', pos_label=1, zero_division=0
        )
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        
        # Acumular para métricas globales
        y_true_all.extend(y_val)
        y_pred_all.extend(y_pred)
        if y_pred_proba is not None:
            y_pred_proba_all.extend(y_pred_proba)
    
    # Métricas promedio
    metrics = {
        'accuracy_mean': np.mean(accuracies),
        'accuracy_std': np.std(accuracies),
        'auc_mean': np.mean(aucs) if aucs else None,
        'auc_std': np.std(aucs) if aucs else None,
        'precision_mean': np.mean(precisions),
        'precision_std': np.std(precisions),
        'recall_mean': np.mean(recalls),
        'recall_std': np.std(recalls),
        'f1_mean': np.mean(f1_scores),
        'f1_std': np.std(f1_scores),
    }
    
    # Métricas globales
    global_metrics = {
        'global_accuracy': accuracy_score(y_true_all, y_pred_all),
        'global_auc': roc_auc_score(y_true_all, y_pred_proba_all) if y_pred_proba_all else None,
        'global_precision': precision_recall_fscore_support(
            y_true_all, y_pred_all, average='binary', pos_label=1, zero_division=0
        )[0],
        'global_recall': precision_recall_fscore_support(
            y_true_all, y_pred_all, average='binary', pos_label=1, zero_division=0
        )[1],
        'global_f1': precision_recall_fscore_support(
            y_true_all, y_pred_all, average='binary', pos_label=1, zero_division=0
        )[2],
    }
    
    return metrics, global_metrics

# Evaluar múltiples modelos
models = {
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'RandomForest': RandomForestClassifier(random_state=42, n_estimators=100),
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
}

results = {}
for name, model in models.items():
    print(f"\nEvaluando {name}...")
    metrics, global_metrics = evaluate_model(model, X, y, cv=5)
    results[name] = {
        'cv_metrics': metrics,
        'global_metrics': global_metrics
    }
    
    # Imprimir resultados
    print(f"  Validación Cruzada (promedio ± std):")
    print(f"    Accuracy:  {metrics['accuracy_mean']:.4f} ± {metrics['accuracy_std']:.4f}")
    if metrics['auc_mean']:
        print(f"    AUC:       {metrics['auc_mean']:.4f} ± {metrics['auc_std']:.4f}")
    print(f"    Precision: {metrics['precision_mean']:.4f} ± {metrics['precision_std']:.4f}")
    print(f"    Recall:    {metrics['recall_mean']:.4f} ± {metrics['recall_std']:.4f}")
    print(f"    F1:        {metrics['f1_mean']:.4f} ± {metrics['f1_std']:.4f}")
    
    print(f"  Métricas Globales (en todos los datos):")
    print(f"    Accuracy:  {global_metrics['global_accuracy']:.4f}")
    if global_metrics['global_auc']:
        print(f"    AUC:       {global_metrics['global_auc']:.4f}")
    print(f"    Precision: {global_metrics['global_precision']:.4f}")
    print(f"    Recall:    {global_metrics['global_recall']:.4f}")
    print(f"    F1:        {global_metrics['global_f1']:.4f}")

# Opcional: Graficar comparación de modelos
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
metrics_to_plot = ['accuracy', 'auc', 'precision', 'recall', 'f1']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 3, idx % 3]
    model_names = list(results.keys())
    means = [results[name]['cv_metrics'][f'{metric}_mean'] for name in model_names]
    stds = [results[name]['cv_metrics'][f'{metric}_std'] for name in model_names]
    
    ax.bar(model_names, means, yerr=stds, capsize=5, alpha=0.7)
    ax.set_title(f'{metric.capitalize()} (CV)')
    ax.set_ylabel('Score')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Optimización de Hiperparámetros

In [ ]:
# Optimización para Random Forest
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'class_weight': ['balanced', {0: 1, 1: 3}]
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(
    rf, 
    param_grid, 
    cv=3,  # Usar menos folds por tiempo
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled_df, y_train)

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor F1-Score:", grid_search.best_score_)

# Evaluar el mejor modelo
best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test_scaled_df)
print("\n=== Mejor Random Forest ===")
print(classification_report(y_test, y_pred_best, target_names=['Normal', 'Anomalía']))

In [ ]:
# Obtener importancia de características del mejor modelo
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': best_rf.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
plt.title('Top 15 Características Más Importantes')
plt.xlabel('Importancia')
plt.tight_layout()
plt.show()

print("Top 10 características más importantes:")
print(feature_importance.head(10))
